In [1]:
# import library
import matplotlib.pyplot as plt
from random import randint, shuffle, sample, uniform

In [2]:
class Chromosome:
    def __init__(self, path, distance):
        self.path = path
        self.distance = distance
        self.prob = 0
        
    def show(self):
        return str(self.path) + '# ' + str(self.distance)
        

In [3]:
def calculateDistance(distance, chromosome):
    d = 0
    path = chromosome.path
    for i in range(len(path) - 1):
        d += distance[path[i]][path[i+1]]
    return d

In [4]:
def generatePath(n=15):
    x = list(range(n))
    shuffle(x)
    return x

In [5]:
def generatePopulation(m=10, n=15):
    populations = []
    while len(populations) < m:
        path = generatePath(n)
        newChromosome = Chromosome(path, 0)
        if newChromosome not in populations:
            populations.append(newChromosome)
    return populations

In [6]:
def calculatePopulationsDistance(data, populations):
    temp_populations = []
    for chromosome in populations:
        distance = calculateDistance(data, chromosome)
        chromosome.distance = distance
        temp_populations.append(chromosome)
    return temp_populations

In [7]:
def crossover(parent1, parent2, N=15):
    start = randint(1, N//2)
    end = randint(N//2 + 1, N-1)
    sub1, sub2 = parent1[start:end], parent2[start:end]
    child1, child2 = parent1, parent2
    parent1 = [x for x in parent1 if x not in sub2]
    parent2 = [x for x in parent2 if x not in sub1]
    child1 = parent2[:start] + sub1 + parent2[start:]
    child2 = parent1[:start] + sub2 + parent1[start:]
#     print(start, end)
    return (child1, child2)

In [8]:
def selection(populations, pop_size):
    sumDistance = sum(1/c.distance for c in populations)
    
    sumProb = 0
    for m in populations:
        fitness = 1/m.distance
        m.prob = sumProb + (fitness / sumDistance)
        sumProb += (fitness/ sumDistance)
    new_pop = []
    for e in range(pop_size):
        numberRandom = uniform(0,1)
        new_pop.append([x for x in populations if x.prob >= numberRandom][0])
        
    return new_pop
    

In [18]:
def mutate(chromosome):
    path = chromosome.path
    point1, point2 = sample(range(len(path)),2)
    path[point1], path[point2] = path[point2], path[point1]
    chromosome.path = path
    return chromosome
    

In [21]:
def GA(data, N=15, pop_size=100, generations=5000):
    populations = generatePopulation(pop_size, N)
    
    
    for gen in range(generations):
        new_populations = []
        bestChromosome = None
        populations = calculatePopulationsDistance(data, populations)
        populations = selection(populations, pop_size)
#         populations = sorted(populations, key=lambda x: x.distance)
        
        for i in range(len(populations) - 1):
            if bestChromosome == None or bestChromosome.distance > populations[i].distance:
                bestChromosome = populations[i]
                new_populations.append(bestChromosome)
                if gen%500 == 0:
                    print("gen",gen, bestChromosome.show())
                
            child1, child2 = crossover(populations[i].path, populations[i+1].path, N)
            chrom1, chrom2 = Chromosome(child1, 0), Chromosome(child2, 0)
#             chrom1 = mutate(chrom1)
#             chrom2 = mutate(chrom2)
            
            new_populations.append(chrom1)
            new_populations.append(chrom2)
            
        populations = new_populations

    populations = calculatePopulationsDistance(data, populations)
    print(sorted(populations, key=lambda x: x.distance)[0].show())
        

In [22]:
data = [
    [0,262,398,172,601,392,185,158,181,353,557,682,249,408,491], 
    [262,0,410,186,512,310,90,324,258,342,475,532,167,258,409], 
    [400,414,0,476,313,209,322,256,220,48,288,598,246,342,253], 
    [161,185,472,0,574,371,158,329,267,404,537,655,228,381,470],
    [594,515,313,577,0,209,423,491,434,331,73,518,336,327,126],
    [392,312,209,374,209,0,221,289,231,155,184,415,145,220,117],
    [186,90,319,158,421,218,0,237,171,251,384,502,75,228,317],
    [158,322,249,330,491,289,235,0,72,234,454,634,196,360,388],
    [181,255,228,265,436,234,167,72,0,179,399,579,140,305,332],
    [155,271,48,416,73,155,263,301,177,0,320,552,187,282,254],
    [557,478,287,540,75,184,387,521,397,320,0,545,311,354,153],
    [676,532,606,649,520,416,495,720,575,555,547,0,437,318,397],
    [249,167,246,228,348,145,75,283,140,187,311,437,0,160,243],
    [408,258,348,372,327,225,230,360,305,282,354,318,162,0,204],
    [473,365,253,473,126,117,320,388,330,253,153,396,244,205,0] 
]
N=12
pop_size=500
generations=3000

GA(data,N , pop_size, generations)

gen 0 [10, 0, 4, 3, 7, 2, 6, 1, 8, 9, 5, 11]# 3732
gen 0 [10, 5, 6, 1, 3, 8, 11, 9, 0, 7, 2, 4]# 2957
gen 0 [2, 5, 0, 3, 8, 9, 4, 10, 7, 6, 1, 11]# 2743
gen 0 [11, 10, 4, 6, 1, 3, 9, 0, 8, 7, 2, 5]# 2591
gen 500 [11, 5, 8, 6, 3, 7, 0, 9, 2, 10, 4, 1]# 2738
gen 500 [2, 9, 4, 6, 8, 7, 0, 3, 1, 11, 5, 10]# 2434
gen 500 [11, 10, 2, 7, 8, 9, 0, 3, 1, 6, 5, 4]# 2370
gen 500 [4, 10, 2, 9, 8, 7, 0, 3, 1, 6, 5, 11]# 1895
gen 1000 [5, 4, 6, 3, 2, 10, 0, 8, 7, 1, 9, 11]# 3576
gen 1000 [5, 4, 10, 9, 3, 1, 0, 7, 2, 8, 6, 11]# 2761
gen 1000 [4, 2, 9, 10, 1, 3, 0, 7, 8, 6, 5, 11]# 2536
gen 1000 [10, 4, 9, 1, 6, 3, 0, 8, 2, 7, 5, 11]# 2455
gen 1500 [2, 11, 6, 3, 5, 8, 0, 7, 9, 4, 10, 1]# 3050
gen 1500 [4, 10, 8, 6, 2, 9, 0, 3, 7, 5, 1, 11]# 2793
gen 1500 [11, 1, 6, 10, 7, 3, 0, 8, 5, 2, 9, 4]# 2763
gen 1500 [10, 9, 5, 8, 2, 7, 0, 3, 1, 6, 4, 11]# 2734
gen 1500 [11, 4, 10, 5, 8, 6, 0, 7, 2, 9, 1, 3]# 2273
gen 1500 [10, 4, 11, 5, 8, 7, 0, 3, 1, 6, 9, 2]# 2216
gen 2000 [4, 2, 7, 6, 3, 9, 0, 11, 1, 8, 10,

In [349]:
data = [
    [0,262,398,172,601,392,185,158,181,353,557,682,249,408,491], 
    [262,0,410,186,512,310,90,324,258,342,475,532,167,258,409], 
    [400,414,0,476,313,209,322,256,220,48,288,598,246,342,253], 
    [161,185,472,0,574,371,158,329,267,404,537,655,228,381,470],
    [594,515,313,577,0,209,423,491,434,331,73,518,336,327,126],
    [392,312,209,374,209,0,221,289,231,155,184,415,145,220,117],
    [186,90,319,158,421,218,0,237,171,251,384,502,75,228,317],
    [158,322,249,330,491,289,235,0,72,234,454,634,196,360,388],
    [181,255,228,265,436,234,167,72,0,179,399,579,140,305,332],
    [155,271,48,416,73,155,263,301,177,0,320,552,187,282,254],
    [557,478,287,540,75,184,387,521,397,320,0,545,311,354,153],
    [676,532,606,649,520,416,495,720,575,555,547,0,437,318,397],
    [249,167,246,228,348,145,75,283,140,187,311,437,0,160,243],
    [408,258,348,372,327,225,230,360,305,282,354,318,162,0,204],
    [473,365,253,473,126,117,320,388,330,253,153,396,244,205,0] 
]
p1, p2 = [11, 8, 6, 3, 1, 0, 7, 5, 2, 9, 4, 10], [5, 2, 9, 4, 10, 11, 1, 0, 7, 8, 6, 3]
child1, child2 = crossover(p1, p2, 15)
child1, child2 = Chromosome(child1, 0), Chromosome(child2, 0)
child1.distance = calculateDistance(data, child1)
child2.distance = calculateDistance(data, child2)
print(child1.show(), child2.show())

1 13
[11, 8, 6, 3, 1, 0, 7, 5, 2, 9, 4, 10]# 2197 [5, 2, 9, 4, 10, 11, 1, 0, 7, 8, 6, 3]# 2297


In [14]:
sample([1,2,3,4,5,6,7,8,9,0,11,12,13,14,15,16], k=10)
x = [1,2,3,4,5,6,7,89]
x[:5]
for i in range(0, (len(x) -1), 2):
    print(i,i+1)


0 1
2 3
4 5
6 7
